In [69]:
import ticktack                 # For the compilation of the model
import ode                      # As a method of solving the odes
import numpy as np              # For the basic functions that need to be written
from jax import numpy as jnp    # annoying shit that I wish I didn't hav to deal with 
import jax.experimental.ode as jax  # Jax odeint 
import scipy.integrate as sci   # Scipy odeint 
import pandas as pd             # Becuase I want to 
from time import process_time   # For timing the different ode implementations

In [5]:
parameters = np.zeros(6)            # Empty array to hold the parameters of the prodcution function 
parameters[0] = 7.044873503263437   # Mean of the sinusoidal production 
parameters[1] = 0.18                # Amplitude of the sinusoidal production 
parameters[2] = 11.0                # Period of the sinusoidal production 
parameters[3] = 1.25                # Phase of the sinusoidal production 
parameters[4] = 120.05769867244142  # Height of the super-gaussian 
parameters[5] = 12.0                # Width of the super-gaussian 

In [6]:
projection = np.zeros(11)   # An array to hold the projection of the production function 
projection[0] = 0.7         # Proportion projection into the stratosphere       
projection[1] = 0.3         # Proportion projection into the troposphere

In [7]:
def production(t):    
    """
    The production function defined as the sum of sinusoidal and super-gaussian terms 
    """
    return parameters[0] * (1 + parameters[1] * \
        jnp.sin(2 * jnp.pi / parameters[2] * t + parameters[3])) + \
        parameters[4] * jnp.exp(- (parameters[5] * (t - 775)) ** 16)

In [8]:
cbm = ticktack.load_presaved_model("Guttler14", production_rate_units="atoms/cm^2/s")
cbm.compile()   # Constructing the transfer operator 

In [9]:
u0 = np.array([135.76261605786132, 709.7591911307035,
    1191.489526709938, 3.97158546945107, 45158.65854589925,
    155.54703228960028, 634.290736403387, 423.5954241095565,
    1808.3343542055652, 665.933052175064, 7348.751626918585])

In [10]:
def dydx(y, t):
    """
    The gradient of the carbon box model.
    """
    return cbm._matrix @ y + production(t) * projection

So the plan is to profile all of the ode solvers using a loop through the different namespaces that provide the ode implementations. Basically, ode_namespace = [sci, jax, ode]. I can then use a pandas data frame to store the results in what is basically a fancy dictionary.

In [108]:
odes = {ode: np.NaN, sci: np.NaN, jax: np.NaN}  # For the storage of the profiles 

for namespace in odes:      # Iterating through the namespaces that provide the odeint function 
    timer = process_time()  # starting a timer 
    C14 = namespace.odeint(dydx, u0, np.arange(760.0, 790.0, 1))    # runner the model 
    timer = process_time() - timer  # Stopping the timer 
    odes[namespace] = C14[:, 2] # Storing the result


In [ ]:
import matplotlib.pyplot as plt # Plotting library for visualisations

plt.plot(np.arange(760.0, 790.0, 1), troposphere) # Plotting the entire data 
plt.show()                      # Showing the plot 